<a href="https://colab.research.google.com/github/Sundaynot/Big_data_project/blob/main/Big_data_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CARICAMENTO spark

In [5]:
!pip install pyspark
!pip install -U -q PyDrive2
#l'output 'xxx is not a symbolic link' non influirà sull'implementazione o esecuzione
#per risolvere 'xxx is not a symbolic link', commenta le righe che iniziano con !mv xxxx
#dovresti sostituire xxx.11 con la versione corretta se si verificano altri errori dopo un aggiornamento di Colab
#per ottenre la versione corretta, usa !ls /usr/local/lib per verificarla
#!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
#!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
#!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
#!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
#!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
#!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3
# !sudo ldconfig
#Se si verifica un errore relativo all'esecuzione sopra indicata, puoi provare a commentare le 12 righe sopra sotto l'installazione di
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#l'output 'xxx is not a symbolic link' non influirà sulla tua implementazione o esecuzione
#per risolvere 'xxx is not a symbolic link', puoi commentare le righe che iniziano con !mv xxxx
#dovresti sostituire xxx.11 con la versione corretta se si verificano altri errori dopo un aggiornamento di Colab
#per ottenre la versione corretta, usa !ls /usr/local/lib per verificarla

ln: failed to create symbolic link '/usr/local/lib/libtbbmalloc_proxy.so.2': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbmalloc.so.2': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbbind_2_5.so.3': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbb.so.12': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbbind_2_0.so.3': File exists
ln: failed to create symbolic link '/usr/local/lib/libtbbbind.so.3': File exists
openjdk-8-jdk-headless is already the newest version (8u462-ga~us1-0ubuntu2~22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


Creazione della SparkSession


In [7]:

# 4. Creazione della SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("LordOfTheRings_Analysis") \
    .getOrCreate()

print("✅ Ambiente Spark pronto!")


✅ Ambiente Spark pronto!


CLONAZIONE DEL REPOSITORY GIT

In [9]:

# ===============================================================
# FASE 2: CLONAZIONE DEL REPOSITORY GIT
# ===============================================================

# Nome della cartella e URL del repo
repo_name = "Big_data_project"
repo_url = "https://github.com/sundaynot/Big_data_project.git"

# 🔁 Clona il repo solo se non esiste già
if not os.path.exists(repo_name):
    print(f"📥 Clonazione del repository '{repo_name}'...")
    !git clone {repo_url}
else:
    print(f"✅ Repository '{repo_name}' già presente.")

✅ Repository 'Big_data_project' già presente.


Carica il file

In [10]:
from pyspark.sql.functions import col, lower, regexp_replace, explode, split, concat_ws, monotonically_increasing_id, lag, when
from pyspark.sql.window import Window

# 1. Carica il file direttamente in un DataFrame
# Spark crea un DataFrame con una colonna "value" che contiene ogni riga del file.
df = spark.read.text("/content/Big_data_project/The Lord Of The Ring 1-The Fellowship Of The Ring_djvu.txt")

# 2. Identifica le righe che definiscono un nuovo capitolo
# Creiamo una nuova colonna "chapter_title" che contiene il titolo solo se la riga è un'intestazione di capitolo.
df_with_titles = df.withColumn("chapter_title",
    when(
        col("value").rlike(r"^(Prologue|Chapter\s*\d+\s*\.\s*.*)"),  # match "Prologue" o "1. ..."
        col("value")
    ).otherwise(None)
)

ASSEGNO LO STESSO TITOLO ALLE RIGHE APPARTENENTI ALLO STESSO CAPITOLO

In [17]:
from pyspark.sql.functions import last

# Finestra ordinata per ID (cumulativa, dal primo record a quello corrente)
window_spec = Window.orderBy("id").rowsBetween(Window.unboundedPreceding, 0)

# Propaga l'ultimo titolo valido (non null) verso il basso
df_propagated = df_ordered.withColumn(
    "chapter",
    last(col("chapter_title"), ignorenulls=True).over(window_spec)
)

print("✅ Titoli propagati correttamente:")
#df_propagated.select("id", "value", "chapter_title", "chapter").show(50, truncate=50)




✅ Titoli propagati correttamente:
+---+--------------------------------------------------+-------------+-------+
| id|                                             value|chapter_title|chapter|
+---+--------------------------------------------------+-------------+-------+
|  0|                           "THE LORD OF THE RINGS"|         NULL|   NULL|
|  1|                                                  |         NULL|   NULL|
|  2|                                         Part One |         NULL|   NULL|
|  3|                                                  |         NULL|   NULL|
|  4|                       THE FELLOWSHIP OF THE RING |         NULL|   NULL|
|  5|                                                  |         NULL|   NULL|
|  6|                                    J.R.R.ToIkien |         NULL|   NULL|
|  7|                                                  |         NULL|   NULL|
|  8|                                                  |         NULL|   NULL|
|  9|             

RAGGRUPPA MANTENENDO L'ORDINE

In [21]:

from pyspark.sql.functions import col, collect_list, concat_ws, lower, regexp_replace, sort_array, struct
# Filtra solo le righe non di titolo
df_text = df_propagated.filter(col("chapter_title").isNull())

# Raggruppa e conserva l’ordine con struct(id, value)
chapter_df = (
    df_text.groupBy("chapter")
    .agg(
        sort_array(collect_list(struct(col("id"), col("value")))).alias("lines")
    )
)
chapter_df.show()

# Ricostruisci il testo ordinato
chapter_df = chapter_df.withColumn(
    "text",
    concat_ws(" ", col("lines.value"))
)
chapter_df.show()
# Pulisci il testo finale (minuscolo, senza punteggiatura)
cleaned_df = chapter_df.withColumn(
    "text",
    lower(regexp_replace(col("text"), r"[^a-zA-Z\s]", ""))
)

# 5️⃣ Mostra il risultato
print("✅ Testo raggruppato e pulito per capitolo:")
cleaned_df.select("chapter", "text").show(truncate=80)


+--------------------+--------------------+
|             chapter|               lines|
+--------------------+--------------------+
|                NULL|[{0, "THE LORD OF...|
|Chapter 1 . A Lon...|[{917, }, {918, W...|
|Chapter 2 . The S...|[{2066, }, {2067,...|
|Chapter 3 . Three...|[{3284, }, {3285,...|
|Chapter 4 . A Sho...|[{4445, }, {4446,...|
|Chapter 5 . A Con...|[{5119, }, {5120,...|
|Chapter 6. The Ol...|[{5727, }, {5728,...|
|Chapter 7 . In th...|[{6448, }, {6449,...|
|Chapter 8 . Fog o...|[{7053, }, {7054,...|
|Chapter 9 . At th...|[{7795, }, {7796,...|
|Chapter 10. Strider |[{8543, }, {8544,...|
|Chapter 11. A Kni...|[{9233, }, {9234,...|
|Chapter 12 . Flig...|[{10343, }, {1034...|
|Chapter 1 . Many ...|[{11334, }, {1133...|
|Chapter 2 . The C...|[{12425, }, {1242...|
|Chapter 3 . The R...|[{14180, }, {1418...|
|Chapter 4 . A Jou...|[{15413, }, {1541...|
|Chapter 5 . The B...|[{16730, }, {1673...|
|Chapter 6 . Lothl...|[{17349, }, {1735...|
|Chapter 7 . The M...|[{18441, }

TOKENIZZO

In [27]:
from pyspark.ml.feature import Tokenizer

# Dividi il testo in parole
tokenizer = Tokenizer(inputCol="text", outputCol="words_raw")
tokenized_df = tokenizer.transform(cleaned_df)

# Anteprima
tokenized_df.select("chapter", "words_raw").show(truncate=80)


+-----------------------------------------+--------------------------------------------------------------------------------+
|                                  chapter|                                                                       words_raw|
+-----------------------------------------+--------------------------------------------------------------------------------+
|                                     NULL|[the, lord, of, the, rings, , part, one, , , the, fellowship, of, the, ring, ...|
|       Chapter 1 . A Long-expected Party |[, when, mr, bilbo, baggins, of, bag, end, announced, that, he, would, shortl...|
|      Chapter 2 . The Shadow of the Past |[, , , the, talk, did, not, die, down, in, nine, or, even, ninety, nine, days...|
|            Chapter 3 . Three is Company |[, , , you, ought, to, go, quietly, and, you, ought, to, go, soon, said, gand...|
|    Chapter 4 . A Short Cut to Mushrooms |[, , , in, the, morning, frodo, woke, refreshed, he, was, lying, in, a, bower...|


RIMUOVO LE STOP WORDS

In [29]:
from pyspark.ml.feature import StopWordsRemover

# Rimuove parole comuni come "the", "and", "of"
remover = StopWordsRemover(inputCol="words_raw", outputCol="words")
clean_words_df = remover.transform(tokenized_df)

# Anteprima
clean_words_df.select("chapter", "words").show(truncate=80)


+-----------------------------------------+--------------------------------------------------------------------------------+
|                                  chapter|                                                                           words|
+-----------------------------------------+--------------------------------------------------------------------------------+
|                                     NULL|[lord, rings, , part, one, , , fellowship, ring, , , jrrtoikien, , , , forewo...|
|       Chapter 1 . A Long-expected Party |[, mr, bilbo, baggins, bag, end, announced, shortly, , celebrating, eleventy,...|
|      Chapter 2 . The Shadow of the Past |[, , , talk, die, nine, even, ninety, nine, days, second, , disappearance, mr...|
|            Chapter 3 . Three is Company |[, , , go, quietly, go, soon, said, gandalf, two, , three, weeks, passed, sti...|
|    Chapter 4 . A Short Cut to Mushrooms |[, , , morning, frodo, woke, refreshed, lying, bower, made, , living, tree, b...|


 VEDO QUANTE PAROLE CI SONO IN OGNI CAPITOLO

In [31]:
from pyspark.sql.functions import size, col

# Conta quante parole “pulite” ci sono in ogni capitolo
word_count_df = clean_words_df.withColumn("word_count", size(col("words")))

# Mostra il risultato
word_count_df.select("chapter", "word_count").show(truncate=80)



+-----------------------------------------+----------+
|                                  chapter|word_count|
+-----------------------------------------+----------+
|                                     NULL|      5188|
|       Chapter 1 . A Long-expected Party |      5913|
|      Chapter 2 . The Shadow of the Past |      6265|
|            Chapter 3 . Three is Company |      5765|
|    Chapter 4 . A Short Cut to Mushrooms |      3471|
|       Chapter 5 . A Conspiracy Unmasked |      3038|
|               Chapter 6. The Old Forest |      3882|
|Chapter 7 . In the House of Tom Bombadil |      3316|
|     Chapter 8 . Fog on the Barrow-Downs |      3928|
|              Chapter 9 . At the Sign of |      3679|
|                     Chapter 10. Strider |      3337|
|         Chapter 11. A Knife in the Dark |      5611|
|         Chapter 12 . Flight to the Ford |      5165|
|               Chapter 1 . Many Meetings |      5311|
|       Chapter 2 . The Council of Elrond |      9019|
|         

LE 10 PIUù RICORRENTI

In [32]:
from pyspark.sql.functions import explode, row_number
from pyspark.sql.window import Window

# 2a. Esplode la lista di parole in righe singole
words_exploded_df = clean_words_df.withColumn("word", explode(col("words")))

# 2b. Conta quante volte appare ogni parola per capitolo
word_freq_df = words_exploded_df.groupBy("chapter", "word").count()

# 2c. Ordina e prendi le 10 parole più frequenti per capitolo
window_spec = Window.partitionBy("chapter").orderBy(col("count").desc())
top_words_df = word_freq_df.withColumn("rank", row_number().over(window_spec)) \
                           .filter(col("rank") <= 10)

# Mostra il risultato
top_words_df.select("chapter", "word", "count").show(50, truncate=50)



+-------------------------------------------+---------+-----+
|                                    chapter|     word|count|
+-------------------------------------------+---------+-----+
|                                       NULL|         |  949|
|                                       NULL|  hobbits|   53|
|                                       NULL|     many|   43|
|                                       NULL|    shire|   39|
|                                       NULL|     long|   37|
|                                       NULL|    still|   30|
|                                       NULL|     book|   28|
|                                       NULL|     ring|   27|
|                                       NULL|     time|   27|
|                                       NULL|    bilbo|   27|
|         Chapter 1 . A Long-expected Party |         | 1192|
|         Chapter 1 . A Long-expected Party |    bilbo|   81|
|         Chapter 1 . A Long-expected Party |     said|   74|
|       

In [33]:
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml import Pipeline

# 1️⃣ Calcolo del Term Frequency (TF)
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000)

# 2️⃣ Calcolo dell'Inverse Document Frequency (IDF)
idf = IDF(inputCol="rawFeatures", outputCol="features")

# 3️⃣ Creazione della pipeline
pipeline = Pipeline(stages=[hashingTF, idf])

# 4️⃣ Addestramento del modello TF-IDF
tfidf_model = pipeline.fit(clean_words_df)

# 5️⃣ Trasformazione dei dati
tfidf_df = tfidf_model.transform(clean_words_df)

print("📊 Calcolo TF-IDF completato. Ogni capitolo è ora un vettore di 'features'.")
tfidf_df.select("chapter", "features").show(truncate=50)


📊 Calcolo TF-IDF completato. Ogni capitolo è ora un vettore di 'features'.
+-----------------------------------------+--------------------------------------------------+
|                                  chapter|                                          features|
+-----------------------------------------+--------------------------------------------------+
|                                     NULL|(10000,[7,15,17,23,24,29,30,32,42,50,52,55,57,6...|
|       Chapter 1 . A Long-expected Party |(10000,[7,15,20,23,24,26,30,32,37,50,52,55,58,6...|
|      Chapter 2 . The Shadow of the Past |(10000,[0,7,15,17,23,24,27,30,32,33,44,52,55,60...|
|            Chapter 3 . Three is Company |(10000,[6,7,15,18,20,23,24,26,37,50,52,53,55,63...|
|    Chapter 4 . A Short Cut to Mushrooms |(10000,[15,17,20,24,41,47,52,55,80,88,94,122,15...|
|       Chapter 5 . A Conspiracy Unmasked |(10000,[7,15,24,30,48,50,52,55,60,63,78,80,88,1...|
|               Chapter 6. The Old Forest |(10000,[7,20,23,24,31,41,42